# Extended Kalman filter
This is an implementation of the example Kalman filter: [ExEKF.m](https://github.com/cybergalactic/MSS/blob/master/mssExamples/ExEKF.m).

ExEKF Discrete-time extended Kalman filter (EKF) implementation demonstrating
how the "predictor-corrector representation" can be applied to the
nonlinear model:

$dx_1/dt = x_2$ <br>
$dx_2/dt = a * x_2 * abs(x_2) + b * u + white noise $ <br>
$y = x_1 + white noise$ <br>

The GNSS position measurement frequency f_gnss [Hz] can be chosen smaller or
equal to the  sampling frequency f_s [Hz]. The ratio between the 
frequencies must be an integer:

Integer:  Z = f_s/f_gnss >= 1 

Author:    Thor I. Fossen <br>
Date:      17 Oct, 2018 <br>
Revisions: 28 Feb. 2020, minor updates of notation <br>


In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
import sympy as sp

import src.visualization.book_format as book_format
book_format.set_style()
from src.substitute_dynamic_symbols import lambdify
from sympy import Matrix

In [ ]:
x_1,x_2, a, b, u, w, h = sp.symbols("x_1,x_2, a, b, u, w, h")
jac = sp.eye(2) + Matrix([x_2,
       a * x_2 * x_2 + b * u + w]).jacobian([x_1,x_2])*h
jac

In [ ]:
lambda_jacobian = lambdify(jac)

In [ ]:
lambda_jacobian(a=1,h=0.1,x_2=3)

In [ ]:
from src.kalman_filter import extended_kalman_filter_example, simulate_model

In [ ]:
# simulation parameters
N = 100  # no. of iterations
f_m = 1  # sampling frequency [Hz]
h_m = 1 / f_m  # sampling time: h  = 1/f_s (s)
t = np.arange(0, N * h_m, h_m)

# initial values for x
x0 = np.array([[0, 0]]).T
us = 0.1 * np.sin(0.1 * t)  # inputs
np.random.seed(42)
ws = 0.1 * np.random.normal(scale=1, size=N)  # process noise

df = simulate_model(x0=x0, us=us, ws=ws, t=t)

fig, axes = plt.subplots(nrows=3)
df.plot(y="u", label="u (input)", ax=axes[0])
df.plot(y="x_1", ax=axes[1])
df.plot(y="x_2", ax=axes[2])
plt.show()


In [ ]:
## Measured yaw angle:
df["epsilon"] = 0.1 * np.random.normal(scale=3, size=N)  # measurement noise
df["y"] = df["x_1"] + df["epsilon"]
ys = np.zeros((N, 1))  # 1!
ys[:, 0] = df["y"].values

In [ ]:
## Discretisized system matrixes:
f_s = 10  # sampling frequency [Hz]
h = 1 / f_s  # sampling time: h  = 1/f_s (s)
    
# initialization of Kalman filter
x0 = np.array([[3.5, 0]]).T
P_prd = np.diag([1, 1])
Qd = 1
Rd = 10
df2 = extended_kalman_filter_example(
        x0=x0, P_prd=P_prd, h_m=h_m, h=h, us=us, ys=ys, Qd=Qd, Rd=Rd
    )

In [ ]:
h

In [ ]:
df2.head()

In [ ]:
fig,axes=plt.subplots(nrows=3)
df.plot(y='u', label='u (input)', ax=axes[0])
axes[1].set_ylabel('$x_1$')
df.plot(y='y', style='.', alpha=0.7, ax=axes[1])
df.plot(y='x_1', label='model', ax=axes[1])
df2.plot(y='x_1 predictor', label='predictor', style='--', ax=axes[1])
df2.plot(y='x_1', label='kalman', style=':', ax=axes[1])


axes[2].set_ylabel('$x_2$')
df.plot(y='x_2', label='model', ax=axes[2]);
df2.plot(y='x_2 predictor', label='predictor', style='--', ax=axes[2]);
df2.plot(y='x_2', label='kalman', style=':', ax=axes[2])